In [16]:
import pandas, random, math
import matplotlib.pyplot as plt

In [10]:
# random.seed(1)
mean_demand = random.randint(100, 200)
noise = [random.randint(math.ceil(-mean_demand/2), math.floor(mean_demand/2)) for _ in range(360)]

In [11]:
daily_demand = [mean_demand+noise[i] for i in range(360)]

In [26]:
demand_df = pandas.DataFrame(data={'demand': daily_demand})

In [27]:
demand_df

demand
0       148
1        59
2        85
3       125
4       153
..      ...
355      55
356     108
357     149
358      77
359     109

[360 rows x 1 columns]

In [28]:
# x_values = [i for i in range(360)]
# plt.figure(figsize=(12,8))
# plt.plot(x_values, daily_demand)
# plt.plot(x_values, [mean_demand for _ in range(360)])

In [29]:
# pred_horizon = 5
# con_horizon = 1
# N = 3
# 
# demand_predictions = []
# 
# # Loop through demand data to calculate moving average and project it
# for i in range(len(daily_demand) - N + 1):
#     # Calculate the moving average for the last N steps
#     moving_avg = demand_df['demand'].iloc[i:i+N].mean()
#     
#     # Create a list of predictions for pred_horizon steps ahead
#     predictions = [moving_avg] * pred_horizon
#     
#     # Append this prediction set to the main list
#     demand_predictions.append(predictions)
# 
# # Convert to DataFrame for better visualization if needed
# predictions_df = pandas.DataFrame(demand_predictions, columns=[f'Pred_{i+1}' for i in range(pred_horizon)])
# 
# print(predictions_df)

         Pred_1      Pred_2      Pred_3      Pred_4      Pred_5
0     97.333333   97.333333   97.333333   97.333333   97.333333
1     89.666667   89.666667   89.666667   89.666667   89.666667
2    121.000000  121.000000  121.000000  121.000000  121.000000
3    116.000000  116.000000  116.000000  116.000000  116.000000
4    102.666667  102.666667  102.666667  102.666667  102.666667
..          ...         ...         ...         ...         ...
353   61.000000   61.000000   61.000000   61.000000   61.000000
354   75.666667   75.666667   75.666667   75.666667   75.666667
355  104.000000  104.000000  104.000000  104.000000  104.000000
356  111.333333  111.333333  111.333333  111.333333  111.333333
357  111.666667  111.666667  111.666667  111.666667  111.666667

[358 rows x 5 columns]


In [31]:
test_demand_list = [random.randint(1,10) for i in range(10)]
test_demand_list

[8, 2, 8, 2, 2, 5, 7, 10, 9, 5]

In [55]:
ph = 5
ch = 1
N = 3
f_dict =dict()
init_forecast = test_demand_list[0]

for t in range(0, len(test_demand_list), ch):
    # print(t)
    flist = []

    if t==0:      
        f_dict.update({t: {f'F({t},{i})': init_forecast for i in range(t + 1, t + 1 + ph)}})
    elif t>=N:
        f_dict.update({t: {f'F({t},{i})': sum(test_demand_list[j] for j in range(t-1, t-N-1, -1))/N for i in range(t+1, t+ph+1)}})



    # flists.append(flist)

In [56]:
f_dict

{0: {'F(0,1)': 8, 'F(0,2)': 8, 'F(0,3)': 8, 'F(0,4)': 8, 'F(0,5)': 8},
 3: {'F(3,4)': 6.0,
  'F(3,5)': 6.0,
  'F(3,6)': 6.0,
  'F(3,7)': 6.0,
  'F(3,8)': 6.0},
 4: {'F(4,5)': 4.0,
  'F(4,6)': 4.0,
  'F(4,7)': 4.0,
  'F(4,8)': 4.0,
  'F(4,9)': 4.0},
 5: {'F(5,6)': 4.0,
  'F(5,7)': 4.0,
  'F(5,8)': 4.0,
  'F(5,9)': 4.0,
  'F(5,10)': 4.0},
 6: {'F(6,7)': 3.0,
  'F(6,8)': 3.0,
  'F(6,9)': 3.0,
  'F(6,10)': 3.0,
  'F(6,11)': 3.0},
 7: {'F(7,8)': 4.666666666666667,
  'F(7,9)': 4.666666666666667,
  'F(7,10)': 4.666666666666667,
  'F(7,11)': 4.666666666666667,
  'F(7,12)': 4.666666666666667},
 8: {'F(8,9)': 7.333333333333333,
  'F(8,10)': 7.333333333333333,
  'F(8,11)': 7.333333333333333,
  'F(8,12)': 7.333333333333333,
  'F(8,13)': 7.333333333333333},
 9: {'F(9,10)': 8.666666666666666,
  'F(9,11)': 8.666666666666666,
  'F(9,12)': 8.666666666666666,
  'F(9,13)': 8.666666666666666,
  'F(9,14)': 8.666666666666666}}

In [57]:
def moving_average_forecast(demand_history, forecast_history, t, period_ahead, N=3):
    """
    Calculates a moving average forecast for period t + period_ahead.

    Parameters:
        demand_history (list): List of past demand values, indexed by time.
        forecast_history (list): List of past forecasts, indexed by time.
        t (int): Current time period.
        period_ahead (int): Number of periods ahead for forecasting.
        N (int): Number of periods to consider for the moving average.

    Returns:
        float: Forecasted value for period t + period_ahead.
    """
    # Ensure enough history for the calculation
    if t < N - 1:
        raise ValueError(f"Not enough demand history to calculate moving average with N={N}")

    # Forecast for period t + period_ahead
    if period_ahead == 1:
        # Standard moving average using the last N demand values
        recent_demand = demand_history[max(0, t - N):t]
        return sum(recent_demand) / len(recent_demand)
    
    elif period_ahead >= 2:
        # For future periods beyond t+1, use the custom rule
        values = []

        # Add Forecast(t, t+1) if available
        if len(forecast_history) >= t:
            values.append(forecast_history[t - 1])  # Forecast(t, t+1)

        # Add last N-1 demand values if needed
        recent_demand = demand_history[max(0, t - (N - len(values))):t]
        values.extend(recent_demand)

        # Ensure we have N values
        if len(values) < N:
            raise ValueError(f"Not enough values to calculate forecast with N={N}")

        # Calculate the average
        forecast_t_tp = sum(values) / N
        return forecast_t_tp

# Example usage:
demand_history = [50, 52, 49, 53, 55]  # Example demand data
forecast_history = [50, 51, 50, 54]     # Example forecast data up to time t-1
t = 4                                   # Current period
period_ahead = 2                        # Forecasting for t+2
N = 3                                   # Number of periods for moving average

forecast_t_tp = moving_average_forecast(demand_history, forecast_history, t, period_ahead, N)
print(f"Forecast for period t+{period_ahead}: {forecast_t_tp}")



Forecast for period t+2: 52.0


In [67]:
def moving_average_forecast(demand_history=None, forecast_history=None, t=0, ph=1, N=3, initial_forecast=0):
    """
    Calculates a list of moving average forecasts for periods t+1 through t+ph.

    Parameters:
        demand_history (list, optional): List of past demand values, indexed by time. Defaults to an empty list.
        forecast_history (list, optional): List of past forecasts, indexed by time. Defaults to an empty list.
        t (int): Current time period.
        ph (int): Number of future periods to forecast (t+1, t+2, ..., t+ph).
        N (int): Number of periods to consider for the moving average.
        initial_forecast (float): Initial forecast value when no data is available.

    Returns:
        list: Forecasted values for periods t+1 through t+ph.
    """
    # Default to empty lists if histories are not provided
    demand_history = demand_history if demand_history is not None else []
    forecast_history = forecast_history if forecast_history is not None else []

    forecasts = []

    for period_ahead in range(1, ph + 1):
        if t == 0:
            # At the start (t=0), return the initial forecast value for all periods
            forecasts.append(initial_forecast)
        else:
            if period_ahead == 1:
                # For t+1, calculate the standard moving average of the last N demand values
                available_demand = demand_history[max(0, t - N):t]
                forecast = sum(available_demand) / len(available_demand) if available_demand else initial_forecast
            else:
                # For t+2 and beyond, use custom rule with Forecast(t, t+1) and demand history
                values = []

                # Add Forecast(t, t+1) if available
                if len(forecast_history) >= t:
                    values.append(forecast_history[t - 1])

                # Add up to the last N-1 demand values
                available_demand = demand_history[max(0, t - (N - len(values))):t]
                values.extend(available_demand)

                # Calculate the average, or use initial forecast if no data
                forecast = sum(values) / len(values) if values else initial_forecast

            forecasts.append(forecast)

    return forecasts

# Example usage:
# Without any demand or forecast history
t = 1                                   # Current period
ph = 5                                  # Forecasting for t+1, t+2, t+3
N = 3                                   # Number of periods for moving average
initial_forecast = 10                  # Initial forecast value when no data is available

# forecast_values = moving_average_forecast(t=t, ph=ph, N=N, initial_forecast=initial_forecast)
# print(f"Forecasts made at period {t} for periods {t+1} through {t+ph}: {forecast_values}")

# With demand and forecast history
demand_history = [8, 2, 8, 2, 2, 5, 7, 10, 9, 5]
forecast_history = [10]
forecast_values = moving_average_forecast(demand_history, forecast_history, t, ph, N, initial_forecast)
print(f"Forecasts made at period {t} for periods {t + 1} through {t+ph}: {forecast_values}")


Forecasts made at period 1 for periods 2 through 6: [8.0, 9.0, 9.0, 9.0, 9.0]


In [73]:
def moving_average_forecast(demand_history, forecast_history, t, ph, N=3, initial_forecast=0):
    """
    Calculates a moving average forecast for periods t+1, t+2, ..., t+ph.

    Parameters:
        demand_history (list): List of past demand values, indexed by time.
        forecast_history (list): List of past forecasts, indexed by time.
        t (int): Current time period.
        ph (int): Number of periods ahead for which forecasts are required.
        N (int): Number of periods to consider for the moving average.
        initial_forecast (float): Initial forecast value if no demand or forecast data is available.

    Returns:
        list: Forecasted values for periods t+1, t+2, ..., t+ph.
    """
    forecasts = []
    
    # If no demand and forecast history is available (e.g., t = 0), return initial_forecast repeated ph times
    if t == 0 or (not demand_history and not forecast_history):
        return [initial_forecast] * ph

    for step in range(1, ph + 1):  # Forecast for t+1, t+2, ..., t+ph
        values_to_average = []

        # Try to use the most recent forecast if available
        if step > 1 and len(forecast_history) >= t + step - 1:
            values_to_average.append(forecast_history[t + step - 2])  # Forecast(t, t+step-1)

        # Include the latest demand values up to N total values
        demand_needed = N - len(values_to_average)
        recent_demand = demand_history[max(0, t - demand_needed):t]
        values_to_average.extend(recent_demand)

        # If we still have fewer than N values, just average what we have
        if len(values_to_average) < N:
            forecast_value = sum(values_to_average) / len(values_to_average)
        else:
            # Otherwise, take the average of the most recent N values
            forecast_value = sum(values_to_average[-N:]) / N

        # Add this forecast to the result list and to the forecast history for future steps
        forecasts.append(forecast_value)
        forecast_history.append(forecast_value)  # Update forecast history

    return forecasts

# Example usage
# Without any demand or forecast history
t = 1                                   # Current period
ph = 5                                  # Forecasting for t+1, t+2, t+3
N = 3                                   # Number of periods for moving average
initial_forecast = 10                  # Initial forecast value when no data is available

# forecast_values = moving_average_forecast(t=t, ph=ph, N=N, initial_forecast=initial_forecast)
# print(f"Forecasts made at period {t} for periods {t+1} through {t+ph}: {forecast_values}")

# With demand and forecast history
demand_history = [8, 2, 8, 2, 2, 5, 7, 10, 9, 5]
# forecast_history = [10]
forecast_values = moving_average_forecast(demand_history, forecast_history, t, ph, N, initial_forecast)
print(f"Forecasts made at period {t} for periods {t + 1} through {t+ph}: {forecast_values}")


Forecasts made at period 1 for periods 2 through 6: [8.0, 6.5, 6.5, 6.5, 6.5]


In [75]:
demand_history[0:1]

[8]

In [79]:
demand_history[N-2:N]

[2, 8]

In [77]:
N

3

In [1]:
204*7.52/(400*1976)

0.0019408906882591092

In [2]:
114 + 19.75*2.9

171.275